### Dataset Details
        The Dataset is a Balanced dataset and there are 2 labels : 0 and 1 which shows that it's a classification problem
        We need to verify whether the text statement in the dataset is positive or negative
            There are only 2 labels : 0, 1
                0 represents Negative
                1 represents Positive

#### Checking for the NVIDIA

In [60]:
!nvidia-smi

Tue Mar  9 13:35:13 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    73W / 149W |   2224MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [61]:
print(len(tf.config.experimental.list_physical_devices("GPU")))

1


#### Importing the required Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

#### Load Dataset

In [4]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Sentiment Analysis_NLP/Train.csv")

In [5]:
df.head()

,text,label
0,I grew up (b. 1965) watching and loving the Th...,0
1,"When I put this movie in my DVD player, and sa...",0
2,Why do people who do not know what a particula...,0
3,Even though I have great interest in Biblical ...,0
4,Im a die hard Dads Army fan and nothing will e...,1


In [6]:
df.shape

(40000, 2)

#### Dataset Analysis

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    40000 non-null  object
 1   label   40000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 625.1+ KB


In [8]:
df.isnull().sum()

text     0
label    0
dtype: int64

In [9]:
df['label'].value_counts()

0    20019
1    19981
Name: label, dtype: int64

#### Checking for Stopwords

In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
from nltk.corpus import stopwords

In [13]:
stop_words = print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

#### Dataset Text clearing

In [14]:
df['text'] = df['text'].apply(lambda x:x.lower())

In [15]:
print(df['text'].iloc[3])

even though i have great interest in biblical movies, i was bored to death every minute of the movie. everything is bad. the movie is too long, the acting is most of the time a joke and the script is horrible. i did not get the point in mixing the story about abraham and noah together. so if you value your time and sanity stay away from this horror.


In [16]:
df['text'] = df['text'].apply(lambda x:re.sub('[^a-zA-Z0-9\s]', "", x))

In [17]:
print(df['text'].iloc[3])

even though i have great interest in biblical movies i was bored to death every minute of the movie everything is bad the movie is too long the acting is most of the time a joke and the script is horrible i did not get the point in mixing the story about abraham and noah together so if you value your time and sanity stay away from this horror


In [18]:
stop_words = set(stopwords.words("english"))

In [19]:
df['text'] = df['text'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop_words)]))

In [20]:
print(df['text'].iloc[3])

even though great interest biblical movies bored death every minute movie everything bad movie long acting time joke script horrible get point mixing story abraham noah together value time sanity stay away horror


In [21]:
max_features = 2000
tokenizer = Tokenizer(num_words = max_features, split = ' ')
tokenizer.fit_on_texts(df["text"].values)

In [22]:
print(df['text'].iloc[3])

even though great interest biblical movies bored death every minute movie everything bad movie long acting time joke script horrible get point mixing story abraham noah together value time sanity stay away horror


In [23]:
seq = tokenizer.texts_to_sequences(df['text'].values)

In [24]:
print(seq[1])

[163, 2, 171, 1696, 1590, 1241, 1236, 2, 8, 20, 2, 731, 6, 12, 207, 486, 86, 68, 122, 610, 1292, 148, 880, 221, 702, 20, 128, 261, 835, 406, 32, 20, 2, 388, 704, 1, 20, 48, 604, 1382, 1406, 880, 550, 739, 1241, 106, 641, 18, 20, 2, 486, 86, 4, 1318, 28, 5, 486, 958, 666, 103, 18, 731, 700, 511, 1574, 606, 16, 84, 724, 5, 178, 564, 577, 12, 90, 129, 9, 21, 6, 564, 60, 326, 577, 9, 153, 128, 1, 204, 598, 32, 20, 2, 21, 155, 343, 602, 645, 720, 592, 2, 304, 128, 5, 2, 60, 26, 46, 240, 2, 69, 227, 1597, 128, 6, 223, 2, 436, 41, 128, 45, 880, 110, 916, 1081, 142]


In [25]:
print(len(seq[1]))

125


In [26]:
seq=  pad_sequences(seq)

In [27]:
print(seq[1])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [28]:
seq.shape

(40000, 755)

In [29]:
y = df['label']

In [30]:
y[3]

0

#### Splitting the dataset in to train and test 

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
X_train, X_test, y_train, y_test = train_test_split(seq, y, test_size = 0.2, random_state =0)

In [33]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(32000, 755) (8000, 755) (32000,) (8000,)


In [34]:
print(X_train[3], y_train[3])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [35]:
embed_dimension = 128

#### Neural Network Model Building

In [36]:
model = Sequential()
model.add(Embedding(max_features, embed_dimension, input_length = seq.shape[1]))
model.add(LSTM(128, dropout = 0.2, recurrent_dropout = 0.2))
model.add(Dense(1, activation = "sigmoid"))

In [37]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 755, 128)          256000    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 387,713
Trainable params: 387,713
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.compile(loss = "binary_crossentropy", optimizer = 'adam',metrics = ['accuracy'])

#### Training the model and checking the accuracy

In [39]:
model.fit(X_train, y_train, batch_size = 100, epochs =10, validation_data= (X_test, y_test))

Epoch 1/10
320/320 [==============================] - 448s 1s/step - loss: 0.5244 - accuracy: 0.7395 - val_loss: 0.3266 - val_accuracy: 0.8664
Epoch 2/10
320/320 [==============================] - 450s 1s/step - loss: 0.2848 - accuracy: 0.8882 - val_loss: 0.3163 - val_accuracy: 0.8706
Epoch 3/10
320/320 [==============================] - 457s 1s/step - loss: 0.2511 - accuracy: 0.8974 - val_loss: 0.3154 - val_accuracy: 0.8696
Epoch 4/10
320/320 [==============================] - 443s 1s/step - loss: 0.2318 - accuracy: 0.9082 - val_loss: 0.3153 - val_accuracy: 0.8694
Epoch 5/10
320/320 [==============================] - 443s 1s/step - loss: 0.2152 - accuracy: 0.9139 - val_loss: 0.3438 - val_accuracy: 0.8645
Epoch 6/10
320/320 [==============================] - 448s 1s/step - loss: 0.2045 - accuracy: 0.9206 - val_loss: 0.3326 - val_accuracy: 0.8622
Epoch 7/10
320/320 [==============================] - 449s 1s/step - loss: 0.1916 - accuracy: 0.9256 - val_loss: 0.3434 - val_accuracy: 0.8655

In [64]:
y_pred = model.predict(X_test)
y_pred = (y_pred>0.5)

In [67]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [68]:
print("Accuracy Score : ", accuracy_score(y_test, y_pred))
print("Confusion Matrix : ", confusion_matrix(y_test, y_pred))

Accuracy Score :  0.858375
Confusion Matrix :  [[3344  597]
 [ 536 3523]]


#### Predicting a random statement

In [40]:
text1 = ["The movie is quite nice. I like it."]
text2 = ["The Food was bad."]

In [41]:
from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [42]:
tokenizer.fit_on_texts(text1)

In [43]:
filtered_texts = []
for word in text1:
  if word not in stop_words:
    filtered_texts.append(word)

In [44]:
filtered_texts

['The movie is quite nice. I like it.']

In [45]:
# Converting the texts to sequence
seq1 = tokenizer.texts_to_sequences(filtered_texts)

In [46]:
seq1

[[2, 82, 217, 5]]

In [47]:
seq1 = pad_sequences(seq1, maxlen= 30)

In [48]:
seq1

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          2,  82, 217,   5]], dtype=int32)

In [49]:
y_pred = model.predict_classes(seq1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [50]:
y_pred

array([[1]], dtype=int32)

In [51]:
tokenizer.fit_on_texts(text2)

In [52]:
filtered_texts2 = []
for word in text2:
  if word not in stop_words:
    filtered_texts2.append(word)

In [53]:
filtered_texts2

['The Food was bad.']

In [54]:
# Converting the texts to sequence
seq2 = tokenizer.texts_to_sequences(filtered_texts2)

In [55]:
seq2

[[1548, 18]]

In [56]:
seq2 = pad_sequences(seq2, maxlen= 30)

In [57]:
y_pred2 = model.predict_classes(seq2)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [58]:
y_pred2

array([[0]], dtype=int32)